In [1]:
#Need to get X and Y
#Going through metrics
import numbers
import glob
import os
import csv

#calculating number of kernels total
total_kernels = []
kernel_count = 0
if True:
    combined_data_ = {}
    target_kernels = []
    
metric_targets = []
all_sig_metrics = []
#bench_targets = [ "gaussianP" , "gaussianV", "gemm", "stream", "kmeans_cudaV", "kmeans_cudaP" ]
#for filen_ in glob.glob("/home/yzamora/power/nvidia_gpus/all_apps/mixed_results/*.csv"):
#for filen_ in glob.glob("/home/yzamora/power/nvidia_gpus/all_apps/gaussian_results/*.csv"):
#for filen_ in glob.glob("/gpfs/jlse-fs0/users/yzamora/v100_testing/v100_p100_results/*.csv"):
#for filen_ in glob.glob("/gpfs/jlse-fs0/users/yzamora/p100_testing/five_apps/*.csv"):
for filen_ in glob.glob("/gpfs/jlse-fs0/users/yzamora/stephanie_gpu_testing/combined/*.csv"):
    filen = os.path.basename(filen_)
    #print (filen)
    filen_split = filen.split('.')[0].split('_')
    bench_name = filen_split[0]
    size = filen_split[1]
    
    #Run if applications contain kmeans in name
    if False: 
        if 'kmeans' in filen_split[0]:
            size = (filen.split('.')[0].split('N')[1])
        else:
            size = filen_split[1].split('N')[1]
    
    #metric_name = filen_split[1]
    #print(filen_split[0][-1]) #gives you p or v
    #size_str = filen_split[1].split('N')[1]
    
    #if not (metric_name in metric_targets): continue
    #if not (bench_name in bench_targets): continue
    
    key_root = bench_name
    #print(key_root)
    levels = ["Idle", "Low","High", "Max"]
    bw_units = ["GB", "MB", "KB" ,"0B"]
    # Now open the file and look for the data
    with open(filen_ ,'r') as file_handle:
        #print (file_handle)
        data_found = False
        ncols = 1
        fdata = csv.reader(file_handle)
        index_lookup = {}
        #print(filen_)
        for line_split in fdata:
            #print (line_split)
            lsplt = (len(line_split) > 0)
      
            if data_found:
                #print("data found")
                if lsplt and len(line_split) == ncols:
                    #percent - strip off end
                    # Get metric name here
                    #mname_index = index_lookup['Metric Name']
                    #metric_name = line_split[ mname_index ]
                    #if not (metric_name in metric_targets): continue
                    
                    # Read in desired value for the current metric
                    target_index = index_lookup['Avg']; value = 0
                    metric_name = line_split[index_lookup['Metric Name']]
                    #print (line_split[target_index].isdecimal())
                    if line_split[target_index].isdecimal():
                        if line_split[target_index]!= '0':
                            #print(line_split[target_index])
                            all_sig_metrics.append(metric_name)
                            value = int(line_split[ target_index ])
                        
                        # Labeled with percentage
                    elif "%" == line_split[target_index][-1]:
                        #print ("percentage loop")
                        all_sig_metrics.append(metric_name)
                        value = float(line_split[ target_index ][0:7]) / 100.0
                        
                    # Labeled with bandwidth units
                    elif line_split[ target_index ][-4:-2] in bw_units:
                        # Just take the first 
                        units = line_split[ target_index ][-4:-2]
                        all_sig_metrics.append(metric_name)
                        mfact = 1.0
                        if   units == "KB": mfact = 1024
                        elif units == "MB": mfact = 1024*1024
                        elif units == "GB": mfact = 1024*1024*1024
                        elif units == "0B":  mfact = 1
                        value = float(line_split[ target_index ][0:7]) * mfact
                    
                    # idle, low, max
                    elif line_split[ target_index ][-1] == ")":
                        #print ("low")
                        all_sig_metrics.append(metric_name)
                        value = int(line_split[ target_index].split('(')[1].split(")")[0])
                        
                    # otherwise, float
                    #elif not(float(line_split[ target_index ]).is_integer()):
                    else:
                        #print(line_split[ target_index ].split('(')[0])
                        #print("in float")
                        #print(line_split[ target_index ].split('(')[0])
                        value = float(line_split[ target_index ])
                    
                    
                    
                     # Parse name of kernel
                    kernel_name = line_split[ index_lookup['Kernel'] ].split('(')[0]
                    if not(kernel_name in total_kernels):
                        total_kernels.append(kernel_name)
                        #print(kernel_name)
                        kernel_count += 1
                    
                        
                    # Define kernel-specific key
                    key = key_root + size + "_" + kernel_name 
                    
                    # Initialize dict for this key, if it is new
                    if not (key in combined_data_): 
                        combined_data_ [ key ] = {}
                    if not (kernel_name in target_kernels):
                        target_kernels.append(kernel_name)
                        #combined_data_ [ key ][ 'size' ] = int( size_str )
                        
                    # Store value for the metric being read right now
                    combined_data_ [key][ metric_name ] = value
                    combined_data_[key]["kernelname"] = kernel_name
                    
                else: data_found = False


            elif lsplt and line_split[0] == 'Device' and line_split[1] == 'Kernel':
                # Set flag that we are at the data:
                data_found = True
                # Set number of columns in table:
                ncols = len(line_split)
                # Generate an index lookup table:
                idx = 0
                for term in line_split:
                    index_lookup[term] = idx
                    idx += 1
                #print(index_lookup)

print(kernel_count)


15


In [2]:
print(len(total_kernels))

15


In [3]:
print(combined_data_['gemmN5216_sgemm_128x128x8_NN_vec'].keys())

KeyError: 'gemmN5216_sgemm_128x128x8_NN_vec'

In [ ]:
#Obtaining y vector
import numpy as np
Y = []
for key, value in combined_data_.items():
    Y.append(combined_data_[key]['ipc'])
    
print(np.array(Y).shape)

In [ ]:
#Obtaining X vector
import os
import sys
import pandas as pd
from sklearn.preprocessing import scale
#print(combined_data_.keys())
df = pd.DataFrame.from_dict(combined_data_,orient='index')
df = df.dropna(axis=1,how='any')
#print(df.shape)
#print(df)
labels = df['kernelname'].tolist()
df2 = df.drop(columns=['kernelname'])
df2_values = df2.values
data = scale(df2.values)
n_samples, n_features = data.shape
print(n_samples, n_features)
X = data

In [ ]:
X[:30,].shape

In [ ]:
array_of_image_vectors = np.asarray(X[0:30,])
print(array_of_image_vectors)

In [ ]:
#Scaling X
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [ ]:
# Compare Algorithms
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import BayesianRidge
#from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.gaussian_process.kernels import RBF

# load dataset
#url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
#names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
#dataframe = pandas.read_csv(url, names=names)
#array = dataframe.values
#X = array[:,0:8]
#Y = array[:,8]
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
models.append(('BR', BayesianRidge(compute_score=True)))
models.append(('KNN', KNeighborsRegressor()))
models.append(('SVM', svm.SVR()))
models.append(('RF', RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100)))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    #cv = KFold(len(y), nfolds)
    #cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
    cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring="neg_mean_squared_error")
    results.append(cv_results)
    names.append(name)
    msg = "%s: Mean squared error: %f (+/- %f) " % (name, cv_results.mean(), cv_results.std())
    print(msg)



# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Model Mean Squared Error')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.savefig("Algo_comp.png")
plt.show()